# NMarkov

- Numerical computation for continuous-time Markov chain (CTMC)
- Partially it can be applied to the analysis of discrete-time Markov chain (DTMC)

## Install

Use Pkg to install the packages. The packages are located in GitHub. Please run the following commands:

- For Julia 1.0.5
```julia
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))
```

In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))

## Initialize

Load `NMarkov`. Also I recommend to load `SparseArrays` to handle the matrix of CTMC.

In [ ]:
using SparseArrays
using NMarkov
using Plots

## Steady-state analysis

### Example1: A dense kernel with GTH algorithm

Solve the stationary distribution of the following CTMC

- State
    - U: System is up
    - D: System is down
- Transition rates
    - U to D: lambda (failure rate)
    - D to U: mu (repair rate)

In [ ]:
# parameters
lambda = 1/100000
mu = 1/10

In [ ]:
# CTMC kernel (Note: diagonal entries are negative)
Q = [
    -lambda lambda;
    mu -mu
]

In [ ]:
# solve the stationary vecotor of Q wih GTH algorithm
# GTH algorithm is appleid only if Q is a dense matrix
gth(Q)

In [ ]:
# error case!
gth(sparse(Q))

### Example 2: A sparse kernel with Gauss-Seidel (GS) algorithm

Solve the stationary distribution of the following CTMC

- State
    - Sn (n = 0, ..., N): The number of customers in the system is n
- Transition rates
    - Sn to Sn+1 (n = 0, ..., N-1): lambda (arrival rate)
    - Sn to Sn-1 (n = 1, ..., N): mu (service rate)

In [ ]:
# parameters
lambda = 1.0
mu = 2.0
N = 10

In [ ]:
using Origin
Q = spzeros(N+1,N+1)
@origin (Q => 0) begin # the indicies of Q start from 0
    Q[0,0] = -lambda
    Q[0,1] = lambda
    for i = 1:N-1
        Q[i,i+1] = lambda
        Q[i,i-1] = mu
        Q[i,i] = -(lambda + mu)
    end
    Q[N,N-1] = mu
    Q[N,N] = -mu
end
Q

#### Run GS algorithm

- Inputs
    - Q: infinitesimal generator
- Outputs
    - x: the stationary vector such that x * Q = 0
    - conv: a bool whether the algorithm is converged or not
    - iter: the number of iterations
    - rerror: an error when the algorithm stops

```julia
x, conv, iter, rerror = stgs(Q)
```

In [ ]:
x, = stgs(Q)

In [ ]:
# error case
stgs(Matrix(Q))

#### Run power method

- Inputs
    - P: uniformed infinitesimal generator; `P, qv = unif(Q)`
- Outputs
    - x: the stationary vector such that x * Q = 0
    - conv: a bool whether the algorithm is converged or not
    - iter: the number of iterations
    - rerror: an error when the algorithm stops

```julia
x, conv, iter, rerror = stpower(P)
```

In [ ]:
P, qv = unif(Q)
x, = stpower(P)

### Example 3: Sensitivity of stationary vector

Consider the first derivative of stationary distribution of the following CTMC with respect to lambda and mu

- State
    - Sn (n = 0, ..., N): The number of customers in the system is n
- Transition rates
    - Sn to Sn+1 (n = 0, ..., N-1): lambda (arrival rate)
    - Sn to Sn-1 (n = 1, ..., N): mu (service rate)

In [ ]:
# parameters
lambda = 1.0
mu = 2.0
N = 10

# infinitesimal generator
using Origin
Q = spzeros(N+1,N+1)
@origin (Q => 0) begin # the indicies of Q start from 0
    Q[0,0] = -lambda
    Q[0,1] = lambda
    for i = 1:N-1
        Q[i,i+1] = lambda
        Q[i,i-1] = mu
        Q[i,i] = -(lambda + mu)
    end
    Q[N,N-1] = mu
    Q[N,N] = -mu
end
Q

In [ ]:
# first derivative matrix of dQ/dlambda
dQ1 = spzeros(N+1,N+1)
@origin (dQ1 => 0) begin
    dQ1[0,0] = -1.0
    dQ1[0,1] = 1
    for i = 1:N-1
        dQ1[i,i+1] = 1.0
        dQ1[i,i] = -1.0
    end
end

# first derivative matrix of dQ/dmu
dQ2 = spzeros(N+1,N+1)
@origin (dQ2 => 0) begin
    for i = 1:N-1
        dQ2[i,i-1] = 1.0
        dQ2[i,i] = -1.0
    end
    dQ2[N,N-1] = 1.0
    dQ2[N,N] = -1.0
end

#### Run GS-type sensitivity algorithm

- Inputs
    - Q: infinitesimal generator
    - pis: stationary vector of Q
    - b: sensitivity vector; b = dQ'*pis where dQ is the first derivative of Q
- Outputs
    - dx: the first derivative of stationary vector
    - conv: a bool whether the algorithm is converged or not
    - iter: the number of iterations
    - rerror: an error when the algorithm stops

```julia
dx, conv, iter, rerror = stsengs(Q, pis, b)
```

In [ ]:
pis, = stgs(Q)
dx1, = stsengs(Q, pis, dQ1'*pis)
dx2, = stsengs(Q, pis, dQ2'*pis)
println(dx1)
println(dx2)

### Example 4: quasi-stationary vector

Consider the quasi-stationary distribution of the following CTMC

- State
    - Sn (n = 0, ...): The number of customers in the system is n
- Transition rates
    - Sn to Sn+1 (n = 0, ...): lambda (arrival rate)
    - Sn to Sn-1 (n = 1, ...): mu (service rate)
- Obtain the probability of the number of customers provided that the number of customers does not exceed N -> quasi-stationary distribution

In [ ]:
# parameters
lambda = 1.0
mu = 2.0
N = 10

# infinitesimal generator
using Origin
Q = spzeros(N+1,N+1)
@origin (Q => 0) begin # the indicies of Q start from 0
    Q[0,0] = -lambda
    Q[0,1] = lambda
    for i = 1:N-1
        Q[i,i+1] = lambda
        Q[i,i-1] = mu
        Q[i,i] = -(lambda + mu)
    end
    Q[N,N-1] = mu
    Q[N,N] = -(lambda+mu) ## there exists the transition rate to SN+1
end
Q

In [ ]:
# exit rate vector that ensures Q*1 + xi*1 = 0
xi = zeros(N+1)
@origin xi=>0 begin
    xi[N]= lambda
end
xi

#### Run GS-type algorithm for quasi-stationary 

- Inputs
    - Q: infinitesimal generator
    - xi: the exit vector that ensures Q*1 + xi*1 = 0
- Outputs
    - x: the quasi-stationary vector such that x * Q = -gam * x
    - gam: a constant as an eigenvalue
    - conv: a bool whether the algorithm is converged or not
    - iter: the number of iterations
    - rerror: an error when the algorithm stops

```julia
x, gam, conv, iter, rerror = qtstgs(Q, xi)
```

In [ ]:
x, gam, = qstgs(Q, xi)

#### Run power method for quasi-stationary 

- Inputs
    - P: uniformed infinitesimal generator; `P, = unif(Q)`
    - xi: the exit vector that ensures Q*1 + xi*1 = 0
- Outputs
    - x: the quasi-stationary vector such that x * Q = -gam * x
    - gam: a constant as an eigenvalue
    - conv: a bool whether the algorithm is converged or not
    - iter: the number of iterations
    - rerror: an error when the algorithm stops

```julia
x, gam, conv, iter, rerror = qtstpower(P, xi)
```

In [ ]:
P, = unif(Q)
x, gam, = qstpower(P, xi)

## Transient analysis

### Example 1

Solve the transient solution of the following CTMC

- State
    - Sn (n = 0, ..., N): The number of customers in the system is n
- Transition rates
    - Sn to Sn+1 (n = 0, ..., N-1): lambda (arrival rate)
    - Sn to Sn-1 (n = 1, ..., N): mu (service rate)
- Initial probability vector
    - The system is empty at time 0

In [ ]:
# parameters
lambda = 1.0
mu = 2.0
N = 10

using Origin
Q = spzeros(N+1,N+1)
@origin (Q => 0) begin # the indicies of Q start from 0
    Q[0,0] = -lambda
    Q[0,1] = lambda
    for i = 1:N-1
        Q[i,i+1] = lambda
        Q[i,i-1] = mu
        Q[i,i] = -(lambda + mu)
    end
    Q[N,N-1] = mu
    Q[N,N] = -mu
end

In [ ]:
# initial probability vector
x0 = zeros(N+1)
@origin x0=>0 begin
    x0[0] = 1.0
end
x0

#### Solve the transient solution

Solve the following matrix exponential with uniformization

```
xt = exp(tr(Q)*t) * x0
```

where `tr` is an transpose operator.

- Inputs
    - Q: infinitesimal generator
    - x0: initial probability vector
    - t: time
    - tr: transpose (:T) or not (:N). The default is :N.
- Output
    - xt: the transient solution; the probability vector at time t

```julia
xt = mexp(Q, x0, 1.0, transpose=tr)
```

In [ ]:
mexp(Q, x0, 1.0, transpose=:T)

#### Solve the transient solution

Solve the following matrix exponential with uniformization

```
xt = exp(tr(Q)*t) * x0
```

where `tr` is an transpose operator.

- Inputs
    - Q: infinitesimal generator
    - x0: initial probability vector
    - t: a vector of time sequence
    - tr: transpose (:T) or not (:N). The default is :N.
- Output
    - xt: the transient solution; the probability vector at time t

```julia
xt = mexp(Q, x0, [0.0, 1.0, 2.0], transpose=tr)
```

In [ ]:
t = LinRange(0.0, 10.0, 100)
mexp(Q, x0, t, transpose=:T)

#### Solve the transient solution

Solve the following matrix exponential with uniformization

```
xt = exp(Q*t) * x0
cxt = int_0^t exp(Q*u) du * x0
```

- Inputs
    - Q: infinitesimal generator
    - x0: initial probability vector
    - t: a vector of time sequence
    - tr: transpose (:T) or not (:N). The default is :N.
- Output
    - xt: the transient solution; the probability vector at time t
    - cxt: value of integration

```julia
xt,cxt = mexpc(Q, x0, 1.0, )
```

In [ ]:
xt,cxt = mexpc(Q, x0, 1.0, transpose=:T)

In [ ]:
t = LinRange(0.0, 10.0, 100)
xt,cxt = mexpc(Q, x0, t, transpose=:T)

#### Solve the transient solution

Solve the expected rewards for a time sequence

```
instantaneous reward: x * exp(Q*t) * r for t = ts
cumulative reward: x * int_0^t exp(Q*u) du * r for t = ts
```

- Inputs
    - Q: infinitesimal generator
    - x: initial probability vector
    - r: reward vector
    - ts: a vector of time sequence
    - forward: coputation with forward (:T) or backward (:N). The default is :T
- Output
    - irwd: instantaneous reward
    - crwd: cumulative reward
    - x: probability vector at ts[end] (forward) or ts[end] (backward)
    - cx: cumulative probability vector at ts[end] (forward) or ts[end] (backward)

```julia
irwd, crwd, x, cx = tran(Q, x, r, [0.0, 1.0, 2.0], forward=:T)
```

In [ ]:
# reward: expected number of cumtomers
@origin r1=>0 begin
    r1 = Float64[i for i = 0:N]
end

In [ ]:
ts = LinRange(0.0, 50.0, 100)
irwd, crwd, xt, cxt = tran(Q, x0, r1, ts, forward=:T)

In [ ]:
plot(ts, irwd)

In [ ]:
# reward: expected number of lost cumtomers
r2 = zeros(Float64, N+1)
@origin r2=>0 begin
    r2[N] = lambda
end

In [ ]:
irwd, crwd, xt, cxt = tran(Q, x0, r2, ts, forward=:T)
plot(ts, [r/t for (t,r) = zip(ts,crwd)])